<a href="https://colab.research.google.com/github/statlib/learn-polars/blob/main/notebooks/polars-expressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade faker hypothesis polars

In [2]:
import numpy as np
import polars as pl
from datetime import datetime
from faker import Faker

def sim_fin_data(n=1000):
    assert n > 0, "Number of companies must be greater than 0."

    fake = Faker()
    rng = np.random.default_rng(seed=123)  # Initialize random number generator
    r = range(n)

    try:
        # Generate company IDs and names
        company_ids = [fake.unique.uuid4() for _ in r]
        company_names = [fake.company() for _ in r]
        filing_dates = [
            fake.date_between(start_date='-1y', end_date='today')
            for _ in range(n)
        ]
        # Generate balance sheet data using lognormal distribution
        total_assets = rng.lognormal(mean=13, sigma=1.0, size=n)
        total_liabilities = np.clip(rng.lognormal(mean=12, sigma=1.0, size=n), 0, total_assets * 0.95)
        total_equity = total_assets - total_liabilities

        # Generate income statement data using lognormal and normal distributions
        revenue = rng.lognormal(mean=12, sigma=1.0, size=n)
        cogs = revenue * np.clip(rng.normal(loc=0.6, scale=0.1, size=n), 0.1, 0.9)
        operating_expenses = revenue * np.clip(rng.normal(loc=0.2, scale=0.05, size=n), 0.05, 0.4)
        interest_expense = total_liabilities * rng.normal(loc=0.05, scale=0.01, size=n)
        tax_expense = revenue * np.clip(rng.normal(loc=0.2, scale=0.02, size=n), 0.1, 0.3)
        net_income = revenue - cogs - operating_expenses - interest_expense - tax_expense

        # Create a Polars DataFrame
        df = pl.DataFrame({
            'company_id': company_ids,
            'company_name': company_names,
            'filing_date': filing_dates,
            'total_assets': total_assets,
            'total_liabilities': total_liabilities,
            'total_equity': total_equity,
            'revenue': revenue,
            'cost_of_goods_sold': cogs,
            'operating_expenses': operating_expenses,
            'interest_expense': interest_expense,
            'tax_expense': tax_expense,
            'net_income': net_income
        })

    except Exception as e:
         return print(f"An error occurred while generating dummy data: {e}")

    return df


In [3]:
df = sim_fin_data()

In [4]:
df.sample(5)

company_id,company_name,filing_date,total_assets,total_liabilities,total_equity,revenue,cost_of_goods_sold,operating_expenses,interest_expense,tax_expense,net_income
str,str,date,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""80b30710-516c-…","""Harris-William…",2022-12-27,99866.722323,15891.606352,83975.115972,105251.91282,42337.554718,25326.398281,680.570424,22425.606777,14481.78262
"""3992b65d-525e-…","""Bridges, Wrigh…",2022-12-26,450361.752091,129083.945462,321277.806629,336090.680403,173981.894263,66231.059329,7033.823597,73504.155893,15339.747321
"""e7d7f13e-1aad-…","""Rogers-Lee""",2023-09-08,54060.412108,51357.391503,2703.020605,269475.195198,144082.05881,29643.919646,1824.700029,62797.72328,31126.793433
"""1411edef-5017-…","""Lopez-Oneal""",2023-07-09,375484.939273,326289.193938,49195.745335,223115.909017,160928.487334,40091.299143,17874.621818,41732.720382,-37511.21966
"""43e36a0a-8022-…","""Douglas, Perez…",2023-01-01,259164.145499,179822.221035,79341.924464,276378.706164,143105.742721,68886.788284,8789.77628,48414.422469,7181.97641


In [5]:
df.filter(
    pl.col("filing_date").is_between(
        datetime(2023, 11, 1),
        datetime(2023, 11, 5)
    )
)

company_id,company_name,filing_date,total_assets,total_liabilities,total_equity,revenue,cost_of_goods_sold,operating_expenses,interest_expense,tax_expense,net_income
str,str,date,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""d88742db-6e76-…","""Nguyen, Landry…",2023-11-02,332605.781684,72553.173849,260052.607835,96919.823861,66729.787868,19255.788092,4148.016195,18569.156819,-11782.925113
"""9ed4258d-e772-…","""Stewart, Wong …",2023-11-02,990791.481009,158695.415865,832096.065144,1.1467e6,663050.028155,267353.030053,6120.651995,240423.164469,-30199.302371
"""f1e28cce-f0bb-…","""Holmes Group""",2023-11-01,1.1741e6,644150.034361,529905.455928,359244.73598,231914.968384,82300.382571,26774.587544,62263.747391,-44008.94991
"""200fef36-f64b-…","""Burch-Benton""",2023-11-01,2.5748e6,1.4096e6,1.1652e6,123778.093125,87938.122834,22921.472843,68567.351139,21823.697495,-77472.551186
"""84832fde-6dcd-…","""Robinson-Sande…",2023-11-02,1.7352e6,298287.589749,1.4370e6,129888.208849,64603.17185,20039.345387,15013.88646,24206.73796,6025.067192
"""49293330-89f1-…","""Burns-Hall""",2023-11-02,205482.917016,195208.771166,10274.145851,104466.680579,49828.894728,18788.446987,6033.176287,20931.960731,8884.201847
"""5440c602-fb02-…","""Berg, Morales …",2023-11-02,765443.07134,403640.836908,361802.234432,735405.542581,571273.348171,118855.020516,16533.454208,150258.973444,-121515.253757
"""71391fab-6386-…","""Rogers, Goodma…",2023-11-03,19063.79901,18110.609059,953.18995,260219.674448,134614.191848,49038.882034,1085.502234,61557.688258,13923.410075
"""2a9125f4-b797-…","""Huber Ltd""",2023-11-02,279712.471091,265726.847537,13985.623555,127300.578707,80758.006332,29225.797056,15694.001385,27212.472823,-25589.698889


In [6]:
df.filter(
    pl.col("filing_date").dt.year() < 2021
)

company_id,company_name,filing_date,total_assets,total_liabilities,total_equity,revenue,cost_of_goods_sold,operating_expenses,interest_expense,tax_expense,net_income
str,str,date,f64,f64,f64,f64,f64,f64,f64,f64,f64


In [8]:
(
    df.with_columns([
        pl.col("filing_date").n_unique().over("company_id").alias('filing_date_count'),
        pl.col("filing_date").dt.quarter().alias("filing_date")
    ])
    .filter(
        pl.col("total_assets") > 5e6
    )
    .melt(
        id_vars="filing_date",
        value_vars=["total_assets", "total_liabilities"]
    )
    .group_by("variable")
    .agg(
        pl.col("value").median()
    )
)

variable,value
str,f64
"""total_assets""",5.8362e6
"""total_liabilit…",226576.470411


In [9]:
df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
print(df)

shape: (5, 4)
┌──────┬───────┬──────────┬────────┐
│ nrs  ┆ names ┆ random   ┆ groups │
│ ---  ┆ ---   ┆ ---      ┆ ---    │
│ i64  ┆ str   ┆ f64      ┆ str    │
╞══════╪═══════╪══════════╪════════╡
│ 1    ┆ foo   ┆ 0.137719 ┆ A      │
│ 2    ┆ ham   ┆ 0.263739 ┆ A      │
│ 3    ┆ spam  ┆ 0.371153 ┆ B      │
│ null ┆ egg   ┆ 0.009119 ┆ C      │
│ 5    ┆ null  ┆ 0.736574 ┆ B      │
└──────┴───────┴──────────┴────────┘


In [10]:
df_numerical = df.select(
    (pl.col("nrs") + 5).alias("nrs + 5"),
    (pl.col("nrs") - 5).alias("nrs - 5"),
    (pl.col("nrs") * pl.col("random")).alias("nrs * random"),
    (pl.col("nrs") / pl.col("random")).alias("nrs / random"),
)
print(df_numerical)

shape: (5, 4)
┌─────────┬─────────┬──────────────┬──────────────┐
│ nrs + 5 ┆ nrs - 5 ┆ nrs * random ┆ nrs / random │
│ ---     ┆ ---     ┆ ---          ┆ ---          │
│ i64     ┆ i64     ┆ f64          ┆ f64          │
╞═════════╪═════════╪══════════════╪══════════════╡
│ 6       ┆ -4      ┆ 0.137719     ┆ 7.261162     │
│ 7       ┆ -3      ┆ 0.527478     ┆ 7.583258     │
│ 8       ┆ -2      ┆ 1.113459     ┆ 8.082918     │
│ null    ┆ null    ┆ null         ┆ null         │
│ 10      ┆ 0       ┆ 3.68287      ┆ 6.788183     │
└─────────┴─────────┴──────────────┴──────────────┘


In [11]:
df_logical = df.select(
    (pl.col("nrs") > 1).alias("nrs > 1"),
    (pl.col("random") <= 0.5).alias("random <= .5"),
    (pl.col("nrs") != 1).alias("nrs != 1"),
    (pl.col("nrs") == 1).alias("nrs == 1"),
    ((pl.col("random") <= 0.5) & (pl.col("nrs") > 1)).alias("and_expr"),  # and
    ((pl.col("random") <= 0.5) | (pl.col("nrs") > 1)).alias("or_expr"),  # or
)
print(df_logical)

shape: (5, 6)
┌─────────┬──────────────┬──────────┬──────────┬──────────┬─────────┐
│ nrs > 1 ┆ random <= .5 ┆ nrs != 1 ┆ nrs == 1 ┆ and_expr ┆ or_expr │
│ ---     ┆ ---          ┆ ---      ┆ ---      ┆ ---      ┆ ---     │
│ bool    ┆ bool         ┆ bool     ┆ bool     ┆ bool     ┆ bool    │
╞═════════╪══════════════╪══════════╪══════════╪══════════╪═════════╡
│ false   ┆ true         ┆ false    ┆ true     ┆ false    ┆ true    │
│ true    ┆ true         ┆ true     ┆ false    ┆ true     ┆ true    │
│ true    ┆ true         ┆ true     ┆ false    ┆ true     ┆ true    │
│ null    ┆ true         ┆ null     ┆ null     ┆ null     ┆ true    │
│ true    ┆ false        ┆ true     ┆ false    ┆ false    ┆ true    │
└─────────┴──────────────┴──────────┴──────────┴──────────┴─────────┘


In [12]:
df.select(
    pl.col("nrs").sort().head(2),
    pl.col("random").filter(pl.col("random") < 1).sum()
)

nrs,random
i64,f64
null,1.518304
1,1.518304


In [22]:
from polars.testing.parametric import dataframes
from polars import NUMERIC_DTYPES
from hypothesis import given

@given(
    dataframes(
        cols=5,
        allowed_dtypes=NUMERIC_DTYPES,
        null_probability=0.1
    )
)
def test_numeric(df):
    assert all(df[col].is_numeric() for col in df.columns)

In [31]:
from polars.testing.parametric import column, dataframes
from hypothesis.strategies import floats, sampled_from, text
from hypothesis import given

from string import ascii_letters, digits

id_chars = ascii_letters + digits


@given(
    dataframes(
        cols=[
            column("id", strategy=text(min_size=4, max_size=4, alphabet=id_chars)),
            column("ccy", strategy=sampled_from(["GBP", "EUR", "JPY", "USD"])),
            column("price", strategy=floats(min_value=0.0, max_value=1000.0)),
        ],
        min_size=5,
        lazy=True,
    )
)
def test_price_calculations(lf):
    assert len(lf.collect()['id']) == 4
    # print(lf.collect())

In [24]:
from polars.testing.parametric import create_list_strategy, dataframes, column
from hypothesis.strategies import composite
from hypothesis import given


@composite
def uint8_pairs(draw, uints=create_list_strategy(pl.UInt8, size=2)):
    pairs = list(zip(draw(uints), draw(uints)))
    return [sorted(ints) for ints in pairs]


@given(
    dataframes(
        cols=[
            column("colx", strategy=uint8_pairs()),
            column("coly", strategy=uint8_pairs()),
            column("colz", strategy=uint8_pairs()),
        ],
        size=3,
    )
)

def test_miscellaneous(df):


SyntaxError: ignored